In [116]:
#import twitterscraper
import praw
import numpy as np
import pandas as pd
import tweepy
import datetime as dt
import nltk
import gensim
from textblob import TextBlob
from collections import Counter
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier
import re, string, random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/divyamrana/nltk_data...


True

In [3]:
sid = SentimentIntensityAnalyzer()

In [137]:
df = pd.read_csv('/Users/divyamrana/Documents/Research Data/Training dataset/cleaned_training.csv', encoding='latin-1')

In [138]:
df.get('Emotion Score').value_counts()

4    800000
0    799999
Name: Emotion Score, dtype: int64

In [139]:
df = df.sample(frac=1)
df = df.head(3000)

In [140]:
df.shape[0]

3000

In [141]:
df = df.fillna('')
df = df[df.get('cleaned_text') != '']

In [142]:
df['Vader'] = df['Tweet'].apply(lambda cleaned: sid.polarity_scores(cleaned))

In [143]:
df['compound']  = df['Vader'].apply(lambda score_dict: score_dict['compound'])

In [144]:
df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c >=0 else 'neg')

In [145]:
df

,Unnamed: 0,Unnamed: 0.1,Emotion Score,Tweet,cleaned_text,Vader,compound,comp_score
581415,581415,581415,0,Ughh these double days blow.. Too bad it goes ...,ughh double days blow too bad goes summer,"{'neg': 0.241, 'neu': 0.759, 'pos': 0.0, 'comp...",-0.5423,neg
864536,864536,864536,4,good morning my dear friends! and what a lovel...,good morning dear friends lovely day too sunny...,"{'neg': 0.0, 'neu': 0.452, 'pos': 0.548, 'comp...",0.9531,pos
468507,468507,468507,0,@SandiMon its a never ending cycle with our ki...,ending cycle kids winter better week starting ...,"{'neg': 0.0, 'neu': 0.879, 'pos': 0.121, 'comp...",0.4404,pos
1460087,1460087,1460087,4,@shiftin so we usually chat about the practice...,usually chat practices tools but it s approach...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
1019087,1019087,1019087,4,@b50 @gulpanag is 100% genuine.. pata chal jat...,genuine pata chal jata hai n yes reply too,"{'neg': 0.0, 'neu': 0.798, 'pos': 0.202, 'comp...",0.5080,pos
...,...,...,...,...,...,...,...,...
157380,157380,157380,0,has headache and stomachache,headache stomachache,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,pos
346384,346384,346384,0,morning early birds. on the way to work on an ...,morning early birds way work stomache gonna long,"{'neg': 0.101, 'neu': 0.899, 'pos': 0.0, 'comp...",-0.2023,neg
1412250,1412250,1412250,4,@T_Billa It sounds like you have your work cut...,it sounds like work cut you i know achieve att...,"{'neg': 0.101, 'neu': 0.766, 'pos': 0.134, 'co...",0.1759,pos
902417,902417,902417,4,@Karen230683 good good!!,good good,"{'neg': 0.0, 'neu': 0.135, 'pos': 0.865, 'comp...",0.7494,pos


In [146]:
df.get('Emotion Score').value_counts()

0    1530
4    1457
Name: Emotion Score, dtype: int64

In [147]:
def prec(vader, original):
    if (original == 4) & (vader == 'pos'):
        return True
    elif (original == 0) & (vader == 'neg'):
        return True
    else:
        return False
    

In [148]:
prec('pos', 0)

False

In [149]:
x = np.vectorize(prec)(df['comp_score'], df['Emotion Score'])

In [150]:
x = pd.Series(x)

In [154]:
x.value_counts()

True     1963
False    1024
dtype: int64

In [155]:
x.value_counts()[1] / (x.value_counts()[0] + x.value_counts()[1])

0.6571811181787747